This notebook generates a series of LTL formula and associated traces from the resultant automaton DAG.

*NOTE*  

This notebook requires Python 3.10 or later to run.


In [1]:
import random
import sys 
import re
import dataclasses
import copy 

import spot
import spot.gen as sg

import ply.lex as lex
import ply.yacc as yacc

import ipywidgets as widgets

import pandas as pd

from pathlib import Path

from graphviz import Digraph
from graphviz import Graph
from graphviz import Source   # Used to read existing dot files

from IPython.display import HTML, display

spot.setup(size='12,8')

---
### The Lexer

See the docs at:   
https://ply.readthedocs.io/_/downloads/en/latest/pdf/


In [2]:
class DotLexer(object):
    states = (
      ('htmlstring','exclusive'),
      ('htmlstyle','exclusive'),
    )

    reserved = {
                'arrowhead'   : 'ARROWHEAD',
                'arrowsize'   : 'ARROWSIZE',
                'center'      : 'CENTER',
                'digraph'     : 'DIGRAPH',
                'edge'        : 'EDGESTYLE',
                'fillcolor'   : 'FILLCOLOR',
                'fontname'    : 'FONTNAME',
                'height'      : 'HEIGHT',
                'label'       : 'LABEL',
                'labelloc'    : 'LABELLOC',
                'node'        : 'NODESTYLE',
                'rankdir'     : 'RANKDIR',
                'size'        : 'SIZE',
                'shape'       : 'SHAPE',
                'style'       : 'STYLE',
                'peripheries' : 'PERIPHERIES',
                'width'       : 'WIDTH',
               }
    
    literals = ['(',')','=',',','{','}']
        
    tokens = [ 
                'CONNECTOR',
                'EOL',
                'HTML_START',
                'HTML_END',
                'ID',
                'NUMBER',
                'STRING',
                'LBRACKET',
                'RBRACKET',
              ]  + list(reserved.values())
    
    t_CONNECTOR = r'->'
    t_NUMBER    = r'(\.|[0-9])[0-9]*'
    
    #A string containing ignored characters (spaces and tabs)
    t_ANY_ignore  = ' \t\r,'
    
    def __init__(self):
        # Build the lexer
        self.lexer = lex.lex(module=self,debug=0,reflags=re.UNICODE | re.VERBOSE)
        self.htmlLevel = 0

    def t_EOL(self,t):
        r'(;\n|\n|;)'
        t.value = t.value
        return t
        
    def t_INITIAL_htmlstyle_ID(self,t):
        r'[a-zA-Z_!&|\u0080-\u3000/][a-zA-Z_!&|0-9\.\u0080-\u3000/]*'
        t.type = self.reserved.get(t.value,'ID')    # Check for reserved words
        return t
        
    def t_ANY_HTML_START(self,t):
        r'<'
        t.value = t.value
        self.htmlLevel += 1
        if self.htmlLevel == 1:
            t.lexer.begin('htmlstring')
        else:
            t.lexer.begin('htmlstyle')
        # print(t.lexer.lexdata)
        # print(t.lexer.lexdata[t.lexer.lexpos:])
        # lineRem = re.search('[^;\n]*[;\n|\n|;]',t.lexer.lexdata[t.lexer.lexpos:]).group()
        # candidate = '/' + (re.search('<([^>]*)>',t.lexer.lexmatch.group()).group(1))
        # print(f'lineRem: {lineRem}, {type(lineRem)}')
        # print(f'candidate: {candidate}, {type(candidate)}')
        # check = re.search(candidate,lineRem)
        # print(f'Is {candidate} in {lineRem}: {check}')
        return t

    def t_htmlstring_htmlstyle_HTML_END(self,t):
        r'>'
        t.value = t.value
        self.htmlLevel -= 1
        if self.htmlLevel >= 1:
            t.lexer.begin('htmlstring')
        else:
            t.lexer.begin('INITIAL')
        return t
        
    def t_htmlstring_STRING(self,t):
        r'[^><]+'
        t.value = t.value
        return t

    def t_LBRACKET(self,t):
        r'\['
        t.value = t.value
        return t

    def t_RBRACKET(self,t):
        r'\]'
        t.value = t.value
        return t
        
    def t_INITIAL_htmlstyle_STRING(self,t):
        r'(\"[^\"]*\")|(\'[^\']*\')|(\[[^\]]*\])'
        t.value = t.value
        return t
        
    # # Error handling rule
    def t_ANY_error(self,t):
        print("Illegal character '%s'" % t.value[0])
        print(t.value[0].encode('raw_unicode_escape'))
        t.lexer.skip(1)

    # Build the lexer
    def build(self,**kwargs):
        self.lexer = lex.lex(module=self, **kwargs)

    # Test it output
    def test(self,data):
        self.lexer.input(data)
        while True:
            tok = self.lexer.token()
            if not tok:
                break
             
#            print(tok)

            pattern = '.*\(([^,]*),.*'

            matches = re.finditer(pattern, f'{tok}')
            for match in matches:
#                print(f'{match.group(1)}')     
                print(f'{tok}, {match.group(1)}')     
                    
        if self.htmlLevel != 0:
            print(f'Error in HTML start/end tags: {"Missing End Tag" if self.htmlLevel > 0 else "Too many End Tags"}')


----

### The Parser

In [205]:
class DotParser(object):

    tokens = DotLexer.tokens
    literals = DotLexer.literals
    reserved= DotLexer.reserved

    def __init__(self):
        self.lexer = DotLexer()
        self.parser = yacc.yacc(module=self,write_tables=False)

    def parse(self,dot):
        return self.parser.parse(dot, debug=True)
        
    def p_graph(self,p):
        ''' graph : DIGRAPH ID body
                  | DIGRAPH STRING body
                  '''
        p[0] = Node(p[1],[p[2]],[p[3]])
#        p[0] = f'{p[1]} {p[2]} {p[3]}'
    #    print(f'graph:       {p[0]}')
        print(f'Self')
    
    def p_body(self,p):
        '''body : bodystart statements bodyend
                '''
        p[0] = Node("Graph",p[2],p[3])
    #    print(f'Body:        {p[0]}')
        print(f'Body')
    
    def p_bodystart(self,p):
        '''bodystart : '{' 
                     | '{' EOL
                '''
        print(f'Bodystart')
    
    def p_bodyend(self,p):
        '''bodyend : '}' 
                     | '}' EOL
                '''
        print(f'Bodystart')

    def p_statements(self,p):
        '''statements : statements statement
                      | statement
                      '''
        if len(p) == 3:
            print(p[1])
            print(p[1][0].children)
            print(p[2])
            (p[1][0].children).append(p[2])
            print(p[1])
            p[0] = list(p[1])
        else:
            p[0] = [Node("Node",[p[1]],[])]
        print(f'statements {p[0]}')

    def p_styles(self,p):
        '''styles : styles style
                  | style
                  '''
        if len(p) == 3:
            print(p[1])
            print(p[1][0].value)
            print(p[2])
            (p[1][0].value).append(p[2])
            print(p[1])
            p[0] = list(p[1])
        else:
            p[0] = [Node("Style",[p[1]],[])]
        print(f'Styles {p[0]}')
            
    # def p_label(self,p):
    #     '''statement : LABEL '=' STRING EOL
    #                  | LABEL '=' html EOL
    #              '''
    #     pass
        
    def p_nodeStyle(self,p):
        '''statement : NODESTYLE LBRACKET styles RBRACKET EOL
                 '''

        p[0] = Node('Nodestyle',p[3],[])
    
    def p_edgeStyle(self,p):
        '''statement : EDGESTYLE LBRACKET styles RBRACKET EOL
                 '''

        p[0] = Node('Edgestyle',p[3],[])
    
    def p_node(self,p):
        '''statement : NUMBER EOL
                     | ID EOL
                     '''
        pass
    
    def p_nodeStyled(self,p):
        '''statement : NUMBER LBRACKET styles RBRACKET EOL
                     | ID LBRACKET styles RBRACKET EOL
                     '''
        pass
    
    def p_nodeConnect(self,p):
        '''statement : NUMBER CONNECTOR NUMBER EOL
                     | NUMBER CONNECTOR ID EOL
                     | ID CONNECTOR NUMBER EOL
                     | ID CONNECTOR ID EOL
                     '''
        pass
        
    def p_nodeConnectStyled(self,p):
        '''statement : NUMBER CONNECTOR NUMBER LBRACKET style RBRACKET EOL
                     | NUMBER CONNECTOR ID LBRACKET style RBRACKET EOL
                     | ID CONNECTOR NUMBER LBRACKET style RBRACKET EOL
                     | ID CONNECTOR ID LBRACKET style RBRACKET EOL
                     '''
        pass

    def p_styleLone(self,p):
        ''' statement : style EOL 
                      | style
                      '''
        pass
        
    def p_style(self,p):
        '''style : LABELLOC    '=' STRING 
                 | SHAPE       '=' ID
                 | SHAPE       '=' STRING
                 | STYLE       '=' ID
                 | STYLE       '=' STRING
                 | FILLCOLOR   '=' ID
                 | FILLCOLOR   '=' STRING
                 | FONTNAME    '=' ID
                 | FONTNAME    '=' STRING
                 | ARROWHEAD   '=' ID
                 | ARROWSIZE   '=' NUMBER
                 | WIDTH       '=' NUMBER
                 | WIDTH       '=' STRING
                 | HEIGHT      '=' NUMBER
                 | HEIGHT      '=' STRING
                 | PERIPHERIES '=' NUMBER
                 | RANKDIR     '=' ID
                 | CENTER      '=' ID
                 | SIZE        '=' STRING
                 | LABEL       '=' STRING
                 | LABEL       '=' html
                 '''
        # print(f'style:       {p[0]}, {bcolors.BLUE}{type(p[0])}{bcolors.ENDC} {len(p)}')
        p[0] = [Node(p[1],[p[3]],[])]
        print(f'style')

    def p_html(self,p):
        '''html : HTML_START htmlstring HTML_END
                '''
        pass 

    def p_htmlstring(self,p):
        '''htmlstring : htmlstring STRING
                      | htmlstring ID
                      | htmlstring htmlstyle
                      | htmlstyle
                      | ID
                      | STRING
                      '''
        print(f'htmlstring')
        
    def p_htmlstyle(self,p):
        '''htmlstyle : HTML_START ID style HTML_END
                     | HTML_START ID HTML_END
                     '''
        pass
        
    # Error rule for syntax errors
    def p_error(self,p):
        print("Syntax error in input!")
        print(f'{p}')


In [206]:
__file__ = "DC_Parser.ipynb"


----
### LTL Tracegen

What follows is all the support code for building an automaton from an equation. The basic flow of the code is as follows:

* Provide an equation. It may be given directly or randomly generated by Spot.
* Spot is used to convert the equation into a Buchi automaton in dot format.
* The dot graph is parsed into an new automaton.
* Randomly walk the automaton and capture the predicate states on the edges taken.

In [207]:
def getSeed(rndSeed=False,printSeed=False):
#    seed = 96996 # generates a long diagram with multiple buchi labels
    seed = 79398 # generates a medium diagram with a graph label containing an & - not accounted for in yacc syntax
#    seed = 64165 # generates a graph title with brackets and html
    
    if rndSeed == True:
        seed = random.randrange(100000)

    if printSeed == True:
        print(f'Seed Used: {seed}')

    return(seed)

----
#### Node Classes

The following classes are to create an binary tree of an equation


In [208]:
class Node:
    def __init__(self,type,value,children):
        self.type = type
        self.value = value
        self.children = children

    def display(self,tabCnt=0):
        print(f'{chr(9) * tabCnt}{self.type}')
        for v in self.value:
            if type(v) is Node:
                v.display(tabCnt=tabCnt+1)
            else:
                print(f'{chr(9) * tabCnt}{v}')
                
        for c in self.children:
            if type(c) is Node:
                c.display(tabCnt=tabCnt+1)
            else:
                print(f'{chr(9) * tabCnt}{c}')
                
    def __repr__(self):
        return(f'Type {self.type}, Value {self.value}, Children {self.children}')

#### Formula Class

This class is the starting point of the process, holding the original formula and methods to print and interact with it.

One can either request a randomly generated formula from Spot, use a predefined one, or provide one as a string.

In [209]:
class Formula:

    def __init__(self,formulaStr,spotFormula,dOptions):
        self.formulaStr = formulaStr
        self.spotFormula = spotFormula
        self.formula = spotFormula.translate(*dOptions)
        self.formulaDot = self.formula.to_str('dot')
        
    @classmethod
    def getRndFormula(cls,dOptions,totalFormulas=10,rndSeed=True):
    
        seed = getSeed(rndSeed,printSeed=True)

        for formula in spot.randltl(3,totalFormulas,seed=seed, tree_size=(1, 40)):
            pass

        return cls(formula,formula,dOptions)

    @classmethod
    def getFixedFormula(cls,dOptions,fNumber=0):

        formula = [
                  '!(g | y )& r & X(r U( !(r|g) & y & X((y) U ((g & !(y | r)) & X(g U (!(g | y )& r))))))',
                  'G(r->X((r)U(y&X((y)U(g)))))',
                  '(p&X(q | (r & G(s))))U(t&F(u))',
                  '((!s & !p & t & r)  U (!r U ((!u & !t & (!p & X(p|s))))))',
                  'G((((p & !q) U((!p & q) | (r & G(s))))U(t &G(F(u) & X u))) & (t -> !p))',
                  '!(g | y )& r & X(r U( !(r|g) & y & X((y) U ((g & !(y | r)) & X(g U (!(g | y )& r))))))',
                  'G(((p & !q) U(!p & q)) U ((G t ) & X u))',
                  '(G !(c1 & c2)) & (GF c1) & (GF c2)',
                  'GF s &  GF p & GF q & GF !t & GF !u & G(t -> X!p)& G(u -> X!q)& G(p -> !t)& G(q -> !u)',
                  'G(!a | ((!b | X(!c U (d & Fe)) | X(c R !d)) U (c | G(!b | X(!c U (d & Fe)) | X(c R !d)))))',
                  'G((q & X F r) -> X(p -> (!r U s)) U r)',
                  'r & (GF r &  GF y & GF g & G(r -> X(g & !r & !y))& G(g -> X(!g & !r & y))& G(y -> X(!g & r & !y)) & G( g | r | y) )',
                  'r & (GF r &  GF y & GF g & G(r -> r U (g & !r & !y))& G(g -> g U(!g & !r & y))& G(y -> y U(!g & r & !y)) & G( g | r | y) )',
                  'r & ( G(r -> (r U g))& G(g -> (g U y)) & G(y -> (y U r)) & G(( g & !r & !y) | (!g & r & !y) | (!g & !r & y)))',
                  '( (( (p -> F( (u & q))) -> (!p -> !q)) & G( (q -> F(s)))) | G( (!t | !u)))',
                  'F(s)&F(p)&F(q)&(p->!t)&(q->!u)',
                  'G((p ∨ (q ∧ r)) -> X((s) U (t ∧ u)))',
                  'G((p & !t) -> X((q -> X(!q U (r & s & t))) U ((!r | !s) & u)))',
                  'G((p & !t) -> X((q -> X(!q U (r & s & t))) U (((!r & !s) | t | u) U (t & u))))',
                  'G((p & !t) -> X((!q & !u) -> X((q <-> (!r & s & t)) U (!s U ((q & !u) | (!q & (r | !t)))))))',
                  'G(F(p & (q | r)) -> X( X(!s)))',
                  'G((p & (q | r)) -> X(X(s)))',
                  'F(G(p -> X(q))) & G(r -> (F(!s)))',
                  'G(p -> (X(q) & G(r -> F(!s))))',
        ]

        return cls(formula[fNumber],spot.formula(formula[fNumber]),dOptions)

    @classmethod
    def setFormula(cls,dOptions,formula=''):

        return cls(formula,spot.formula(formula),dOptions)

    def getStartState(self):
        return self.formula.get_init_state_number()
        
    def showGraph(self,graphPrint,graphFile):
        dot = Source(self.formulaDot)
    
        if graphPrint['Print'] == True:
            display(dot)
    
        if graphPrint['Source'] == True:
            print(self.formulaDot)

        if graphPrint['Stats'] == True:
            display(self.formula.show_storage())
    
        if graphFile['save'] == True:
            dot.render(graphFile['name'], format=graphFile['format'])

    def hierarchyMap(self):
        display(self.spotFormula.show_mp_hierarchy())

    # Return the Manna and Pnueli Class
    def fClass(self):
        return spot.mp_class(self.spotFormula, 'wv')

    def acceptingRun(self):
        print(f'\n Spot generated accepting trace:')
        print(self.formula.accepting_run())

    def latex(self,title,color=''):
        print(f'\n{color}{"-"*30}\n   {title}\n{bcolors.ENDC}')
        print(f'{self.spotFormula:xc}') #See Spot documentation for further formatting: https://spot.lre.epita.fr/tut01.html
        print(f'{color}{"-"*30}')

    def toLaTeX(self,title,color=''):
        latexStr = ''
        dash = False
        for c in self.formulaStr:
            alpha = c.isalpha()
            cap   = c.isupper()
            if alpha == True and cap == True:
#                latexStr += f'\\mathsf{{{c}}}' #script G
                latexStr += f'{c}'
            elif c == '&' or c == chr(0x2227):
                latexStr += f'\\land '
            elif c == '!' or c == chr(0x00AC):
                latexStr += f'\\lnot '
            elif c == '|' or c == chr(0x2228):
                latexStr += f'\\lor '
            elif c == '-':
                dash = True
            elif dash == True:
                if c == '>':
                    latexStr += f'\\limplies'
                else:
                    latexStr += f'-{c}'
                dash = False
            else:
                latexStr += f'{c}'
        print(f'\n{color}{"-"*4} {title} {"-"*4}{bcolors.ENDC}')
        print(f'{latexStr}\n')
        print(f'{color}{"-"*30}{bcolors.ENDC}')
        
    def printSelf(self):

#        print(f'Starting formula:\n{self.formula}')
        display(self.spotFormula)
        
        

----
# Start of the actual program.

To control the execution of this generator, there are the option parameters to set:

#### Specification Related Parameters
`formulaType` This defines where the specification comes from. The options are:
* 'Fixed' Used a fixed formula found in the Formula class.
* 'Random' Ask Spot for a random formula
* 'Given' Give the formula directly
  
`fixedFormulaNum ` If using a fixed formula, this parameter specifies which one. Currently allows selection 0-15  
`rndNumFormulas` If Spot is generating the formula, define how many to generate. The code will attempt to select an interesting one  
`startFormula` The specific formula to use  

`dOptions` All the formulas are sent to Spot to generate an automaton. This paramenter defines what type of automaton to create. The options are `Complete`, `Unambiguous`, `tgba`. See the Spot documentation for more info.

`rndSeed=True` For repeatability, set this to false. when set to True, the seed used is defined in getSeed().  

#### Trace Related Parameters
`runLen` Length of each trace genrated  
`numRuns` Number of traces to generate  

#### Graph Display Control
`path` Defines where saved files will go  

`graphPrint` Print the original automaton graph  
`graphSource`Save the graph source in dot format  
`graphFile` A dictionary of graph file output control  

`traceQual` Quality of the generated traces in terms of bad vs good. Options are: `Good` - Generate only good traces (e.g. never enter a trap); `Any` - Generate a mix of good and bad traces  
`tracePrint` Display a detail log of each trace showing predicates along with stating and ending states for each trace step  
`traceGraphPrint` Display the automaton graph color coded for coverate. Green = path or state seen in at least one trace  
`traceSource` Save the graph source in dot format 
`traceGraphFile` A dictionary of graph file output control  


In [210]:
formulaType = 'Given' #'Fixed', 'Random', 'Given'
startFormula = 'G(r->X((r)U(y&X((y)U(g)))))'
startFormula = '(l ↔ p) ∧ F(¬p)'
startFormula = '(l ↔ p) ∧ F(¬p) & G !((l & !p) | (!l & p))'
startFormula = 'G(l ↔ p) ∧ F(¬p)'
startFormula = 'G((t ↔ y) ∧ (t → p))'
startFormula = '(G((l ↔ p) ∧ F(¬p)) ∧ G((t ↔ y) ∧ (t → p)))'
#startFormula = 'G(((l ↔ p) ∧ ((t ↔ y) ∧ (t → p)) ∧ F(¬p)))'
startFormula = '(G((l ↔ p) ∧ F(¬p)) ∧ G((t → (y ∧ X¬y)) ∧ (t → X(b U b)) ∧ (¬(b ∧ t) → X¬b) ∧ (t → p)))'
#startFormula = 'G(l <-> p) & G(t -> (p & y & X (!t & !y))) & F(!p)'
#startFormula = 'G(l <-> p) & G((!y U (p & t & X!y))) & F(!p)'
#startFormula = 'GF r &  GF y & GF g & G(r -> X(g & !r & !y)) & G(g -> X(y & !r & !g))& G(y -> X(r & !g & !y))'
#startFormula = 'G((r -> Xg) ∧ (g -> Xy) ∧ (y -> Xr)) ∧ G(¬(r ∧ y) ∧ ¬(r ∧ g) ∧ ¬(y ∧ g))'
#startFormula = 'G((r -> Xg) ∧ (g -> Xy) ∧ (y -> Xr)) ∧ G(¬(r ∧ y) ∧ ¬(r ∧ g) ∧ ¬(y ∧ g)) ∧ G(r -> XXg ∧ y -> XXg)'
startFormula = 'G((r -> Xg) ∧ ((g & Xg) -> Xy) ∧ (y -> Xr)) ∧ G(¬(r ∧ y) ∧ ¬(r ∧ g) ∧ ¬(y ∧ g))'
#startFormula = 'G((r -> Xg) ∧ (g -> Xr)) ∧ G(¬(r ∧ g))'
#startFormula = 'G(r -> (Xg -> (Xg -> Xr))) ∧ G(¬(r ∧ g)) & G(r | g)'
#startFormula = 'G((p <-> g) & F(!p) & ((r -> X(y & X(!y))) & (y <-> r)) & (r & !escape -> (b U (!b | !p))) & G(!b | !r))'

#startFormula = 'G(a -> X F b)'
#startFormula = 'G(a -> X G ( b -> X F (c -> X F d)))'

# These equations come from "Li, R., Gurushankar, K., Heule, M. J., & Rozier, K. Y. (2023, October). What's in a Name? Linear Temporal Logic Literally Represents Time Lines. 
# In 2023 IEEE Working Conference on Software Visualization (VISSOFT) (pp. 73-83). IEEE."
#startFormula = 'G((a & b) -> X !b)' # Figure 8
#startFormula = 'c & (F G a U X(G b & (!a U F a)))' # Figure 9
#startFormula = 'G((p & X !p) | (!p & X p))' # Figure 10

startFormula = 'G((a U (b U (c)))&(!(a & b) & !(a & c) & !(b & c)))'
#startFormula = 'G((a & X !(a & b) | (!a & X (!a & !b)) | (b & X !(a & b) | (!b & X (!a & !b)))))'
#startFormula = 'G(((a & X (!a & b)) | (b & X (!b & a))) & !(a & b) & !(!a & !b))' # RE (ab)*
#startFormula = 'G(((a & X (!a & b & !c)) | (b & X (!b & !a & c)) | (c & X (!c & !b & a))) & !(a & b & c) & !(!a & !b & !c))' # RE (ab)*
#startFormula = 'G(((a & X (!a &  b & !c & !d)) |(b & X (!a & !b &  c & !d)) |(c & X (!a & !b & !c &  d)) |(d & X ( a & !b & !c & !d)))&!(a & b & c & d)&!(!a & !b & !c & !d))' # RE (abcd)*
#startFormula = 'a & X G((((a & X (!a & b & !c)) | (b & X (!b & !a & c)) | (c & X (!c & !b & a))) & G(!(a & b) & !(a & c) & !(b & c))))' # RE a(abc)* 
#startFormula = 'a & X G(((a -> Xb) & (b -> X(c|d)) & (c -> Xa) & (d -> Xb)) & G(!(a & b) & !(a & c) & !(a & d) & !(b & c) & !(b & d) & !(c & d) & (a | b | c |d)))' # RE a(a|b|c|d)(ab(db)*c)* 

#startFormula = 'G((r -> Xg) ∧ (g -> Xy) ∧ (y -> Xr) & G(r | g | y)) ∧ G(¬(r ∧ y) ∧ ¬(r ∧ g) ∧ ¬(y ∧ g) & (F r & F g & F y)) ' # Basic traffic light
#startFormula = 'G(p <-> g) & F(!p) & (g & t <-> y) & GF(p &( a & Xc)) & GF(p & b) & G((a->!b) & (b->!a))' 
#startFormula = 'G(p <-> g) & F(!p) & GF(p &( a & Xc)) & GF(p & b) & G((a->!b) & (b->!a))' 
#startFormula = 'G(g <-> p) & G(b U (!b | a)) & G(a -> X c) & F(¬p) & G(a->p) & G(b->p) & G(c->p) & GFa & GFb & GFc & GF(p & !a & !b & !c) & G(a -> !b) & G(b->!a)'
#startFormula = 'G((a & b) -> X(a & c)) & G((a & c) -> X(d & !a))'
#startFormula = 'G(((G(!s) & (q & r)) -> (s U X((t & u)))))'
#startFormula = 'G((p | (q& r)) ->  X((s) U ( t& u)))'
#startFormula = 'F(p & q)'
#startFormula = 'p'
#startFormula = 'GF((p | q) & (p -> Xq) & (q -> Xp)) & G!(p & q)'
#startFormula = 'G(F(((X((nsg & ewr) & X((nsy & ewr) & X (nsr & ewg))))) | ((X((ewg & nsr) & X((ewy & nsr) & X (ewr & nsg)))))) & F nsg & F nsy & F ewg & F ewy) & G!(nsr & nsg) & G!(nsr & nsy) & G!(nsg & nsy) & G((ewr & (nsg | nsy)) | (nsr & (ewg | ewy))) & G!(ewr & ewy) & G!(ewr & ewg) &G!(ewg & ewy) '
#startFormula = 'GF((ag R ar)| (bg R br)) & GF ag & GF bg & GF ar & GF br & G!(ag & bg) '
#startFormula = 'GF(r & a & o -> !h) & GF(rbl -> rc & b) & GF(rw -> (r & b & d) & X !rw) & GF rw & G(!rw & rc)'
#startFormula = 'GF((as | (ap & lr)) & X(G!(as | ap) U (vs | (vp & av)))) & G!(ap & vp) & G!(ap & !lr) & G!(vp & !av)'
#startFormula = 'F((p -> G(q))&(r->X(s)))'

rndSeed=False

rndNumFormulas = 10
fixedFormulaNum = 4
dOptions = ['Complete', 'Buchi', 'state-based', 'det','high'] # First option must be one of: 'Complete' 'Unambiguous' 'tgba'

saveDir = 'testRun1'
path = Path(__file__).parent.absolute().joinpath(saveDir)
path.mkdir(parents=True, exist_ok=True)


In [211]:
def buildAutomaton(formula: Formula):

    parser = DotParser()
#    results = parser.parse(formula.formulaDot)

#    formula = dotdata = 'digraph MONA_DFA { rankdir = LR; center = true; size = "7.5,10.5"; edge [fontname = Courier]; node [height = .5, width = .5]; node [shape = doublecircle]; 2; node [shape = circle]; 0; 1; node [shape = box]; init [shape = plaintext, label = ""]; init -> 0; 0 -> 1 [label=<X<font color="#1F78B4"><I><B>y</B>⓿</I>z</font>)>]; 1 -> 2 [label="X"]; 2 -> 2 [label="X"];} '
    results = parser.parse(formula)

    return(results)
    

In [213]:
formulaA ='digraph MONA_DFA { rankdir = LR; center = true; size = "7.5,10.5"; edge [fontname = Courier]; node [height = .5, width = .5]; node [shape = doublecircle]; 2; node [shape = circle]; 0; 1; node [shape = box]; init [shape = plaintext, label = ""]; init -> 0; 0 -> 1 [label=<X<font color="#1F78B4"><I><B>y</B>⓿</I>z</font>)>]; 1 -> 2 [label="X"]; 2 -> 2 [label="X"];} '

if formulaType == 'Random':
    totalFormulas = 10
    formula = Formula.getRndFormula(dOptions,totalFormulas=rndNumFormulas,rndSeed=rndSeed)
elif formulaType == 'Fixed':
    formula = Formula.getFixedFormula(dOptions,fNumber=fixedFormulaNum)
else:
    formula = Formula.setFormula(dOptions,startFormula)

graph = formulaA
#graph = formula.formulaDot

p = re.compile('digraph [^\n;]*[\n;][^\n;]*[\n;][^\n;]*[\n;][^\n;]*[\n;][^\n;]*[\n;][^\n;]*[\n;][^\n;]*[\n;][^\n;]*[\n;][^\n;]*[\n;][^\n;]*[\n;][^\n;]*[\n;][^\n;]*[\n;][^\n;]*')
g = p.match(graph).group() + ';\n}'

print(g)
auto = buildAutomaton(g)

print('\n')
print(auto)
print('\n')
auto.display()



digraph MONA_DFA { rankdir = LR; center = true; size = "7.5,10.5"; edge [fontname = Courier]; node [height = .5, width = .5]; node [shape = doublecircle]; 2; node [shape = circle]; 0; 1; node [shape = box]; init [shape = plaintext, label = ""]; init -> 0;
}
Bodystart
style
statements [Type Node, Value [None], Children []]
style
[Type Node, Value [None], Children []]
[]
None
[Type Node, Value [None], Children [None]]
statements [Type Node, Value [None], Children [None]]
style
[Type Node, Value [None], Children [None]]
[None]
None
[Type Node, Value [None], Children [None, None]]
statements [Type Node, Value [None], Children [None, None]]
style
Styles [Type Style, Value [[Type fontname, Value ['Courier'], Children []]], Children []]
[Type Node, Value [None], Children [None, None]]
[None, None]
Type Edgestyle, Value [Type Style, Value [[Type fontname, Value ['Courier'], Children []]], Children []], Children []
[Type Node, Value [None], Children [None, None, Type Edgestyle, Value [Type Styl

Generating LALR tables
PLY: PARSE DEBUG START

State  : 0
Stack  : . LexToken(DIGRAPH,'digraph',1,0)
Action : Shift and goto state 2

State  : 2
Stack  : DIGRAPH . LexToken(ID,'MONA_DFA',1,8)
Action : Shift and goto state 3

State  : 3
Stack  : DIGRAPH ID . LexToken({,'{',1,17)
Action : Shift and goto state 7

State  : 7
Stack  : DIGRAPH ID { . LexToken(RANKDIR,'rankdir',1,19)
Action : Reduce rule [bodystart -> {] with ['{'] and goto state 6
Result : <NoneType @ 0x1090bdfc0> (None)

State  : 6
Stack  : DIGRAPH ID bodystart . LexToken(RANKDIR,'rankdir',1,19)
Action : Shift and goto state 26

State  : 26
Stack  : DIGRAPH ID bodystart RANKDIR . LexToken(=,'=',1,27)
Action : Shift and goto state 53

State  : 53
Stack  : DIGRAPH ID bodystart RANKDIR = . LexToken(ID,'LR',1,29)
Action : Shift and goto state 83

State  : 83
Stack  : DIGRAPH ID bodystart RANKDIR = ID . LexToken(EOL,';',1,31)
Action : Reduce rule [style -> RANKDIR = ID] with ['rankdir','=','LR'] and goto state 15
Result : <list 

TypeError: 'NoneType' object is not iterable

In [ ]:
print(auto)

In [ ]:
print(formula.formulaDot)


This is a little debug code that allows me to limit the length of the input to the parser so there's less text to churn through after throwing an error. It also helps to isolate the erroneous line.

----
#### Code for testing just the parser

In [ ]:
# Build the lexer
lexer = DotLexer()
lexer.build()
g = graph

# Give the lexer some input
lexer.test(g)
print(g)